# Example to train a model
Using the openstf tasks

In [ ]:
import pandas as pd
import IPython
from openstef.pipeline.train_model import train_model_pipeline
from openstef.pipeline.create_forecast import create_forecast_pipeline
from openstef.data_classes.prediction_job import PredictionJobDataClass

In [ ]:
# define properties of training/prediction. We call this a 'prediction_job'
pj = dict(id=287,
        model='xgb',
        quantiles=[10,30,50,70,90],
        forecast_type="demand",
        lat=52.0,
        lon=5.0,
        horizon_minutes=47*60,
        resolution_minutes=15,
        name="Example",          
        hyper_params={}, # Note, this should become optional
        feature_names=None, # Note, this should become optional
        default_modelspecs=None,
       )
pj=PredictionJobDataClass(**pj)

# Load input data
input_data = pd.read_csv('data/get_model_input_pid_287.csv', index_col='index', parse_dates=True)

# Split in training and forecasting data
train_data = input_data.iloc[:-200,:] # everything except last 200 rows (~ 48 hours)
to_forecast_data = input_data.iloc[:-200,:] # last 200 rows


In [ ]:
train_data.head()

In [ ]:
to_forecast_data.head()

# Train a model
Train a model using the high-level pipeline. Store the model and reports on training proces in ./trained_models

In [ ]:
train_model_pipeline(
    pj,
    train_data,
    check_old_model_age=False,
    mlflow_tracking_uri="./trained_models",
    artifact_folder="./trained_models",
    )

You can find the trained model in ./trained_models, along with reports on the training process

In [ ]:
## Inspect local files
IPython.display.HTML(f"<iframe src=./trained_models/{pj['id']}/Predictor0.25.html width=800 height=400></iframe>"
                     f"<iframe src=./trained_models/{pj['id']}/Predictor47.0.html width=800 height=400></iframe>"
                     f"<iframe src=./trained_models/{pj['id']}/weight_plot.html width=800 height=400></iframe>")